In [1]:
import math
from collections import defaultdict
import random

In [50]:
class Dataset:
    def __init__(self, images_file, labels_file, lines_per_image = 28):
        self.images = []
        self.labels = []
        self.counts_by_label = defaultdict(int)
        self.priors = {}
        with open(images_file) as file:
            EOF = False
            while not EOF:
                image = []
                for i in range(lines_per_image):
                    line = file.readline()
                    if not line:
                        EOF = True
                        break
                    image.append(list(line))
                if EOF:
                    break
                self.images.append(image)
        with open(labels_file) as file:
            for i,label in enumerate(file):
                label = int(label)
                self.labels.append(label)
                self.counts_by_label[label]+=1
    def display(self, i):
        print("".join(map(lambda x: "".join(x),self.images[i])))
    def __len__(self):
        return len(self.labels)
    
    def shuffleData(self):
        order = list(zip(self.labels, self.images))
        random.shuffle(order)
        tempL, tempI = zip(*order) #zip turns them into giant tuples, want in list form
        self.labels = list(tempL)
        self.images = list(tempI)
    
    
class Perceptron:
    def __init__(self, label):
        self.label = label                                
        self.bias = 0 
        self.weightVector = [[0 for i in range(28)] for j in range(28)] #int(random.random() *10)
        self.totalCount = 0
        self.decayed = 0
        
    def display(self):
        print('\n'.join(str(self.weightVector[i]) for i in range(28)))
        
    def updateBias(self):
        self.bias = self.totalCount
    
    def getDecay(self, totalImages):
        return int(totalImages/2000)/10.0
    
    def trainVectorOnCorrect(self, image, totalCount): 
        for i in range(28):
            for j in range(28):
                if(image[i][j] != ' ' and image[i][j] != '\n'):
                    self.weightVector[i][j] += 2#(2 - self.getDecay(totalCount)) #alpha = 2 seems to work best
    def trainVectorOnIncorrect(self, image, totalCount):
        for i in range(28):
            for j in range(28):
                if(image[i][j] != ' ' and image[i][j] != '\n'):
                     self.weightVector[i][j] -= 2#(2 - self.getDecay(totalCount)) 
                        
    def imageEvaluation(self, image):
        likelihood = 0
        for i in range(28):
            for j in range(28):
                if(image[i][j] == ' '):
                    likelihood -= self.weightVector[i][j]
                elif image[i][j] != '\n':
                    likelihood += self.weightVector[i][j]
        return likelihood

In [51]:
def retrainLoop(image, actualLabel, totalCount):
    iterationLimit = 10
    while(True):
        if(iterationLimit < 0):
            break
        chances = [perceptrons[j].imageEvaluation(image) for j in range(10)]
        bestGuess = chances.index(max(chances))
        #print(chances)
        #print("is: " +  str(actualLabel) + " guessed: " + str(bestGuess))
        if bestGuess == actualLabel:
            perceptrons[actualLabel].trainVectorOnCorrect(image, totalCount)
            break
        else :
            perceptrons[actualLabel].trainVectorOnIncorrect(perceptrons[bestGuess].weightVector, totalCount)
            perceptrons[actualLabel].trainVectorOnCorrect(image, totalCount)
        iterationLimit -= 1

In [52]:
trainingData = Dataset("trainingimages", "traininglabels")
testData = Dataset("testimages", "testlabels")

In [56]:
perceptrons = [None]*10
for i in range(10):
    perceptrons[i] = Perceptron(i)


#
# RUNNING ALL EPOCHS TAKES A VERY LONG TIME
#
epochs = 4
totalImages = 0
correctFires = 0
misfires_t = 0
for q in range(epochs):
    print("epoch number: " + str(q))
    correctGuesses = 0
    misfires_e = 0
    trainingData.shuffleData()
    for l in range(10): 
        perceptrons[l].updateBias()
    
    for i in range(len(trainingData.images)):
        totalImages +=1
        if totalImages%50 == 0:
            print("images so far: "+ str(totalImages) + "\nmisfires so far: " + str(misfires_t) + "\ncorrect fires so far: " + str(correctFires))
        currImage = trainingData.images[i]
        actualLabel = trainingData.labels[i]
        for j in range(10):
            activate = perceptrons[j].imageEvaluation(currImage) + perceptrons[j].bias
            if activate > ((q+1)* 1000): 
                if j == actualLabel :
                    correctGuesses +=1
                    correctFires+=1
                    perceptrons[actualLabel].totalCount +=1
                    perceptrons[actualLabel].trainVectorOnCorrect(currImage, totalImages)
                else :
                    misfires_e +=1
                    misfires_t +=1
                    retrainLoop(currImage, actualLabel, 0)
            elif j== actualLabel:
                perceptrons[actualLabel].totalCount +=1
                perceptrons[actualLabel].trainVectorOnCorrect(currImage, totalImages)

    print("accuracy this epoch: " + str(correctGuesses/len(trainingData.images)))
    print("there were " + str(misfires_e) + " perceptrons that misfired this epoch")

epoch number: 0
images so far: 50
misfires so far: 0
correct fires so far: 0
images so far: 100
misfires so far: 34
correct fires so far: 23
images so far: 150
misfires so far: 348
correct fires so far: 72
images so far: 200
misfires so far: 732
correct fires so far: 122
images so far: 250
misfires so far: 1115
correct fires so far: 171
images so far: 300
misfires so far: 1515
correct fires so far: 221
images so far: 350
misfires so far: 1935
correct fires so far: 271
images so far: 400
misfires so far: 2363
correct fires so far: 321
images so far: 450
misfires so far: 2801
correct fires so far: 371
images so far: 500
misfires so far: 3250
correct fires so far: 421
images so far: 550
misfires so far: 3700
correct fires so far: 471
images so far: 600
misfires so far: 4150
correct fires so far: 521
images so far: 650
misfires so far: 4600
correct fires so far: 571
images so far: 700
misfires so far: 5050
correct fires so far: 621
images so far: 750
misfires so far: 5500
correct fires so 

KeyboardInterrupt: 

In [11]:

confusion_matrix_count = [[0 for i in range(10)] for j in range(10)]
totalImages = 0
indices = len(testData.images)
for i in range(indices):
    totalImages +=1
    currImage = testData.images[i]
    chances = [perceptrons[j].imageEvaluation(currImage) for j in range(10)]
    bestGuess = chances.index(max(chances))
    actualLabel = testData.labels[i]
    if bestGuess == actualLabel :
        perceptrons[actualLabel].trainVectorOnCorrect(currImage, totalImages)
        confusion_matrix_count[bestGuess][bestGuess] +=1
    else :
        retrainLoop(currImage, actualLabel, totalImages)
        confusion_matrix_count[actualLabel][bestGuess] +=1


In [48]:
for i in range(10):
    print(confusion_matrix_count[i])
setTotal = 0
setCorrect = 0
for i in range(10):
    correct = confusion_matrix_count[i][i]
    total = sum(confusion_matrix_count[i])
    rate = 100* correct/total
    print("Label: "+ str(i) + " was accurate at a rate of: " + str(rate) + "%" )
    setCorrect += correct
    setTotal += total
setRate = 100* setCorrect/setTotal
print("overall accuracy: " + str(setRate) + "% \n")


[74, 0, 0, 0, 2, 6, 2, 0, 4, 1]
[0, 92, 0, 0, 1, 8, 1, 0, 2, 0]
[2, 3, 81, 2, 3, 2, 3, 2, 4, 1]
[0, 0, 0, 84, 0, 8, 0, 4, 3, 1]
[0, 1, 0, 0, 83, 1, 2, 1, 2, 16]
[0, 0, 0, 9, 2, 72, 0, 2, 3, 4]
[0, 4, 4, 0, 5, 4, 72, 0, 2, 1]
[0, 1, 3, 0, 1, 3, 0, 89, 4, 5]
[1, 1, 1, 8, 1, 4, 0, 2, 81, 6]
[0, 0, 0, 3, 4, 5, 0, 3, 1, 83]
Label: 0 was accurate at a rate of: 83.14606741573034%
Label: 1 was accurate at a rate of: 88.46153846153847%
Label: 2 was accurate at a rate of: 78.64077669902913%
Label: 3 was accurate at a rate of: 84.0%
Label: 4 was accurate at a rate of: 78.30188679245283%
Label: 5 was accurate at a rate of: 78.26086956521739%
Label: 6 was accurate at a rate of: 78.26086956521739%
Label: 7 was accurate at a rate of: 83.9622641509434%
Label: 8 was accurate at a rate of: 77.14285714285714%
Label: 9 was accurate at a rate of: 83.83838383838383%
overall accuracy: 81.42570281124497% 



In [49]:
confusion_matrix = [[0 for i in range(10)] for j in range(10)]
for i in range(10):
    rowTotal = sum(confusion_matrix_count[i])
    for j in range(10):
        confusion_matrix[i][j] = '%.2f'%(confusion_matrix_count[i][j]/rowTotal)

for q in range(10):
    print(confusion_matrix[q])

['0.83', '0.00', '0.00', '0.00', '0.02', '0.07', '0.02', '0.00', '0.04', '0.01']
['0.00', '0.88', '0.00', '0.00', '0.01', '0.08', '0.01', '0.00', '0.02', '0.00']
['0.02', '0.03', '0.79', '0.02', '0.03', '0.02', '0.03', '0.02', '0.04', '0.01']
['0.00', '0.00', '0.00', '0.84', '0.00', '0.08', '0.00', '0.04', '0.03', '0.01']
['0.00', '0.01', '0.00', '0.00', '0.78', '0.01', '0.02', '0.01', '0.02', '0.15']
['0.00', '0.00', '0.00', '0.10', '0.02', '0.78', '0.00', '0.02', '0.03', '0.04']
['0.00', '0.04', '0.04', '0.00', '0.05', '0.04', '0.78', '0.00', '0.02', '0.01']
['0.00', '0.01', '0.03', '0.00', '0.01', '0.03', '0.00', '0.84', '0.04', '0.05']
['0.01', '0.01', '0.01', '0.08', '0.01', '0.04', '0.00', '0.02', '0.77', '0.06']
['0.00', '0.00', '0.00', '0.03', '0.04', '0.05', '0.00', '0.03', '0.01', '0.84']
